In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import ParameterGrid
import joblib

First, we’ll identify a target that we’re trying to predict. Our target will be if the next close price will go up or down tomorrow. If the price went up, the target will be 1.0, and if it went down, the target will be 0.0.

EMA explanation

In [3]:
import yfinance as yf
def prepare_stock_data(stock):
    df = pd.read_json('../data/historical_data/' + stock + '.json', lines=True)
    df1 = pd.read_json('../data/historical_data/S&P500.json', lines=True)
    
    df1 = df1.rename(columns={'Open': 'S&P 500 Open', 'Close': 'S&P 500 Close'})

    # Join dataframes
    df = pd.merge(df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']],
                  df1[['Date', 'S&P 500 Open', 'S&P 500 Close']],
                  on='Date', how='outer')
    
    # Set date as index
    df['Date'] = df['Date'].astype(str).str.split(' ').str[0]
    df = df.set_index('Date')
    
    # Add label
    df['Label'] = df.rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0])['Close']
    
    # Shift one day, we can not use the future to predict the past
    df[['Open', 'High', 'Low', 'Close', 'Volume', 'S&P 500 Open', 'S&P 500 Close']] = df[['Open', 'High', 'Low', 'Close', 'Volume', 'S&P 500 Open', 'S&P 500 Close']].shift(1)
    
    df = df.rename(columns={'Open': 'Prev Open', 'High': 'Prev High', 'Low': 'Prev Low', 
                            'Close': 'Prev Close', 'Volume': 'Prev Volume', 'S&P 500 Open': 'Prev S&P 500 Open', 
                            'S&P 500 Close': 'Prev S&P 500 Close'})
    
    
    # Add sentiment analysis
    
    # Compute Exponential Mobile Average (EMA) for stock close prices
    delta = df['Prev Close'] - df['Prev Open']
    df['10 Days EMA'] = np.round(delta.copy().ewm(span=10, adjust=False).mean(), decimals=3)
    df['5 Days EMA'] = np.round(delta.copy().ewm(span=5, adjust=False).mean(), decimals=3)
    df['3 Days EMA'] = np.round(delta.copy().ewm(span=3, adjust=False).mean(), decimals=3)

    delta = df['Prev S&P 500 Close'] - df['Prev S&P 500 Open']
    df['S&P 500 10 Days EMA'] = np.round(delta.copy().ewm(span=10, adjust=False).mean(), decimals=3)
    df['S&P 500 5 Days EMA'] = np.round(delta.copy().ewm(span=5, adjust=False).mean(), decimals=3)
    df['S&P 500 3 Days EMA'] = np.round(delta.copy().ewm(span=3, adjust=False).mean(), decimals=3)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    # Re order columns
    #df = df[['Date', 'Stock Trend EMA', 'S&P 500 Trend EMA', 'Label']]

    return df

df = prepare_stock_data('AMZN')
df

,Prev Open,Prev High,Prev Low,Prev Close,Prev Volume,Prev S&P 500 Open,Prev S&P 500 Close,Label,10 Days EMA,5 Days EMA,3 Days EMA,S&P 500 10 Days EMA,S&P 500 5 Days EMA,S&P 500 3 Days EMA
Date,,,,,,,,,,,,,,
2017-01-04,757.92,758.76,747.70,753.67,3521066.0,2251.57,2257.83,1.0,-4.250,-4.250,-4.250,6.260,6.260,6.260
2017-01-05,758.39,759.68,754.20,757.18,2510526.0,2261.60,2270.75,1.0,-3.697,-3.237,-2.730,6.785,7.223,7.705
2017-01-06,761.55,782.40,760.26,780.45,5830068.0,2268.18,2269.00,1.0,0.411,4.142,8.085,5.701,5.089,4.263
2017-01-09,782.36,799.44,778.48,795.99,5986234.0,2271.14,2276.98,1.0,2.815,7.305,10.858,5.726,5.339,5.051
2017-01-10,798.00,801.77,791.77,796.92,3446109.0,2273.59,2268.90,0.0,2.107,4.510,4.889,3.832,1.996,0.181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-11,3211.71,3233.23,3126.09,3229.72,4375905.0,4655.34,4670.29,1.0,-10.568,-8.592,-2.488,-6.939,-7.771,-2.961
2022-01-12,3230.00,3327.00,3214.03,3307.24,3115146.0,4669.14,4713.07,0.0,5.397,20.019,37.376,2.310,9.463,20.484
2022-01-13,3331.50,3337.56,3288.34,3304.14,2290851.0,4728.59,4726.35,0.0,-0.559,4.226,5.008,1.483,5.562,9.122


the training and test set have to follow chronological order.

In [4]:
predictors = ['Prev Open', 
              'Prev High', 
              'Prev Low', 
              'Prev Close', 
              'Prev Volume',
              '10 Days EMA', 
              '5 Days EMA', 
              '3 Days EMA',
              'S&P 500 10 Days EMA', 
              'S&P 500 5 Days EMA', 
              'S&P 500 3 Days EMA'
             # 'Prev S&P 500 Open',
             #'Prev S&P 500 Close'
            ]
x_train, x_test, y_train, y_test = train_test_split(df[predictors],
                                                    df[['Label']], test_size=.2,
                                                    shuffle=True, random_state=0)

print('Size of train set: ', x_train.shape)
print('Size of test set: ', x_test.shape)
print('Size of train set: ', y_train.shape)
print('Size of test set: ', y_test.shape)

Size of train set:  (1015, 11)
Size of test set:  (254, 11)
Size of train set:  (1015, 1)
Size of test set:  (254, 1)


In [5]:
grid = {'n_estimators': [200], 'max_depth': [3], 'max_features': [2, 8], 'random_state': [42]}
test_scores = []

rf_model = RandomForestRegressor()

for g in ParameterGrid(grid):
    rf_model.set_params(**g) 
    rf_model.fit(x_train, y_train.values.ravel())
    test_scores.append(rf_model.score(x_test, y_test))

best_index = np.argmax(test_scores)
print(test_scores[best_index], ParameterGrid(grid)[best_index])

-0.007278772248345389 {'random_state': 42, 'n_estimators': 200, 'max_features': 2, 'max_depth': 3}


In [6]:
# Spot-Check Algorithms
classifiers = [
    RandomForestClassifier(n_estimators=200, min_samples_split=200, max_features = 3, random_state=1, max_depth=3),
    XGBClassifier(),
    AdaBoostClassifier(),
    KNeighborsClassifier(3),
   # SVC(kernel="linear", C=0.025),
   # SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5)
]

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ("pca", PCA())
])

numeric_features = predictors

preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
]) 


In [7]:
for classifier in classifiers:  
    
    pipe = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier', classifier)
           ])
    
    # Train the model
    pipe.fit(x_train, y_train.values.ravel())
    
    # Use model to make predictions
    y_pred = pipe.predict(x_test)
    
    # Evaluate the performance
    print("\nTraining ", classifier)
    accuracy = accuracy_score(y_pred, y_test)
    print("Accuracy on test set: ", accuracy)
    print("Metrics per class on test set:")

    print("Confusion matrix:")
    metrics.confusion_matrix(y_test, y_pred)

    print(metrics.classification_report(y_test, y_pred))


Training  RandomForestClassifier(max_depth=3, max_features=3, min_samples_split=200,
                       n_estimators=200, random_state=1)
Accuracy on test set:  0.5393700787401575
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.40      0.11      0.17       111
         1.0       0.56      0.87      0.68       143

    accuracy                           0.54       254
   macro avg       0.48      0.49      0.43       254
weighted avg       0.49      0.54      0.46       254

[20:54:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Edoardo\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Training  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.531496062992126
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.46      0.44      0.45       111
         1.0       0.58      0.60      0.59       143

    accuracy                           0.53       254
   macro avg       0.52      0.52      0.52

In [67]:
# Save the Model to disk
filename = 'classification_model.pkl'
final_pipe = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier', classifiers[1])
           ])
joblib.dump(final_pipe, filename)

['classification_model.pkl']

Evaluate the model using other test set

In [68]:
def predict_stock_value(stock):
    df = prepare_stock_data(stock)
    y_test = df['Label']
    
    # Load the Regression Model
    pipe = joblib.load('classification_model.pkl')
    
    # Use model to make predictions
    y_pred = pipe.predict(df[predictors])

    # Evaluate the performance
    print("\n Evaluating ", pipe['classifier'])
    accuracy = accuracy_score(y_pred, y_test)
    print("Accuracy on test set: ", accuracy)
    print("Metrics per class on test set:")

    print("Confusion matrix:")
    metrics.confusion_matrix(y_test, y_pred)

    print(metrics.classification_report(y_test, y_pred))

In [69]:
predict_stock_value('TSLA')


 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.4830575256107171
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.47      0.77      0.59       601
         1.0       0.52      0.22      0.31       668

    accuracy                           0.48      1269
   macro avg       0.50      0.50      

In [70]:
predict_stock_value('AAPL')


 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.46099290780141844
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.46      1.00      0.63       585
         1.0       0.00      0.00      0.00       684

    accuracy                           0.46      1269
   macro avg       0.23      0.50     

C:\Users\Edoardo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Edoardo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Edoardo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [72]:
predict_stock_value('GOOGL')


 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.9085894405043341
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.93      0.87      0.90       574
         1.0       0.90      0.94      0.92       695

    accuracy                           0.91      1269
   macro avg       0.91      0.91      

In [57]:
predict_stock_value('MSFT')


 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.5697399527186762
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.57      0.17      0.26       568
         1.0       0.57      0.90      0.70       701

    accuracy                           0.57      1269
   macro avg       0.57      0.53      